In [ ]:
# === INSTALL ===
!pip install -q gradio opencv-python mtcnn torch torchvision

# === COPY MODEL ===
!cp '/content/drive/MyDrive/deepfake_best.pth' ./

# === APP ===
import os

app_code = '''
import gradio as gr
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as T
from PIL import Image
import cv2
from mtcnn import MTCNN
import numpy as np

# --- Model ---
class DeepfakeDetector(nn.Module):
    def __init__(self):
        super().__init__()
        resnet = models.resnet18(weights=None)
        in_f = resnet.fc.in_features
        resnet.fc = nn.Sequential(nn.Dropout(0.5), nn.Linear(in_f, 1))
        self.model = resnet
    def forward(self, x):
        return torch.sigmoid(self.model(x))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DeepfakeDetector().to(device)
model.load_state_dict(torch.load('deepfake_best.pth', map_location=device))
model.eval()

# --- Face Detector (hidden) ---
detector = MTCNN()

# --- Preprocessing ---
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# --- Crop Face (background) ---
def crop_face(img_pil):
    img_cv = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
    faces = detector.detect_faces(img_cv)
    if len(faces) == 0:
        return None, "No face detected. Use a clear, frontal photo."
    box = max(faces, key=lambda x: x['box'][2] * x['box'][3])['box']
    x, y, w, h = [max(0, v) for v in box]
    face = img_pil.crop((x, y, x+w, y+h))
    return face, None

# --- Predict ---
def predict(img):
    if img is None:
        return "<h3 style='color: #666; text-align: center;'>Upload a photo to begin</h3>"

    face, error = crop_face(img)
    if error:
        return f"<h3 style='color: #d97706; text-align: center;'>{error}</h3>"

    tensor = transform(face).unsqueeze(0).to(device)
    with torch.no_grad():
        prob = model(tensor).item()

    label = "FAKE" if prob >= 0.5 else "REAL"
    conf = prob if label == "FAKE" else 1 - prob
    color = "#dc143c" if label == "FAKE" else "#10b981"

    result_html = f"""
    <div style="
        text-align: center;
        padding: 35px;
        background: linear-gradient(135deg, #f8f9fa, #e9ecef);
        border-radius: 25px;
        box-shadow: 0 10px 30px rgba(0,0,0,0.12);
        margin: 25px 0;
        border: 2px solid {color};
    ">
        <h1 style="color: {color}; font-size: 3.2rem; margin: 0; font-weight: bold;">
            {label}
        </h1>
        <p style="font-size: 1.5rem; color: #1f2937; margin: 12px 0;">
            Confidence: <strong style="color: #1f2937; font-weight: bold;">{conf:.1%}</strong>
        </p>
        <p style="color: #4b5563; font-size: 1.1rem; margin: 8px 0;">
            Model: ResNet18 | Test Accuracy: <strong style="color: #1f2937; font-weight: bold;">96.53%</strong>
        </p>
    </div>
    """

    return result_html

# --- CLEAN & PERFECT UI ---
with gr.Blocks(theme=gr.themes.Soft(), title="Deepfake Detector") as demo:
    gr.HTML("""
    <div style="text-align: center; padding: 35px; background: linear-gradient(135deg, #667eea, #764ba2); border-radius: 25px; color: white; margin-bottom: 30px;">
        <h1 style="font-size: 3rem; margin: 0; font-weight: 800;">Deepfake Detector</h1>
        <p style="font-size: 1.3rem; margin: 10px 0 0; opacity: 0.9;">
            Trained on 10,000 face images • 96.53% Accuracy
        </p>
    </div>
    """)

    input_img = gr.Image(type="pil", label="Upload a Photo", height=420)
    output_html = gr.HTML()

    input_img.change(predict, inputs=input_img, outputs=output_html)

    gr.HTML("""
    <div style="text-align: center; margin-top: 50px; color: #6b7280; font-size: 0.95rem;">
        Face detection runs in background • No data stored • Built with Gradio
    </div>
    """)

demo.launch(share=True, debug=False)
'''

# Write file
with open('app.py', 'w') as f:
    f.write(app_code)

# Run
!python app.py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 71.6 MB/s eta 0:00:00
2025-11-09 13:49:23.226167: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762696163.240788     573 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762696163.245117     573 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762696163.256105     573 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762696163.256148     573 computation_placer.cc:177] computation placer already registered. Plea